In [34]:
import FinanceDataReader as fdr
import yfinance as yf
import pandas as pd
import numpy as np
import warnings
import pickle, time, os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# 모든 컬럼 출력
pd.set_option('display.max_columns', None)
# 모든 행 출력
pd.set_option('display.max_rows', None)
# 열 너비 무제한(문자열 길어도 줄바꿈 없이 출력)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.1f}'.format)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# 영문 컬럼명 -> 한국어 해석
# 재무제표 영문 → 한국어 매핑 (손익/자산/부채/자본 통합)
# 재무제표 영문 → 한국어 매핑 (손익/자산/부채/자본 통합)
FIN_COL_MAP_KR_BS = {
    # 주식/자본
    "Treasury Shares Number": "자사주 수",
    "Ordinary Shares Number": "보통주 수",
    "Share Issued": "발행주식 수",  ## 발행주식수가 줄어들면 소각했다는 의미
    "Common Stock": "보통주 자본금",
    "Preferred Stock": "우선주 자본금",
    "Additional Paid In Capital": "주식발행초과금",
    "Retained Earnings": "이익잉여금", ## 회사가 이익을 꾸준히 내고 있다는 증거, 그 이익을 내부에 재투자하고 있다는 의미, 자본이 증가해 재무적으로 안정적이 됨
    "Treasury Stock": "자사주 금액",
    "Stockholders Equity": "총자본/주주지분(비지배제외)", ## 모기업(지배기업) 소유주에게 귀속되는 부분만 말함
    "Total Equity Gross Minority Interest": "총자본/주주지분(비지배포함)",

    # 부채
    "Total Liabilities Net Minority Interest": "총부채(비지배제외)",
    "Total Non Current Liabilities Net Minority Interest": "비유동부채(비지배제외)",
    "Long Term Debt": "장기차입금",
    "Current Debt": "단기차입금",
    "Current Liabilities": "유동부채",
    "Accounts Payable": "매입채무",

    # 자산
    "Total Assets": "총자산",
    "Total Non Current Assets": "비유동자산",
    "Current Assets": "유동자산",
    "Cash And Cash Equivalents": "현금 및 현금성자산",
    "Inventory": "재고자산",
    "Accounts Receivable": "매출채권",
    "Net PPE": "순유형자산",
    "Goodwill": "영업권",

    # 자본 관련 조정
    "Tangible Book Value": "유형자산 기반 순자산",
    "Invested Capital": "투자자본",
    "Total Capitalization": "총자본(부채+자본)",
    "Other Equity Adjustments": "기타 자본조정",
}


FIN_COL_MAP_KR_YF = {
    "Tax Effect Of Unusual Items": "비정상 항목 세금영향",
    "Tax Rate For Calcs": "계산용 세율",
    "Normalized EBITDA": "정상화 EBITDA",
    "Total Unusual Items": "총 비정상 항목",
    "Total Unusual Items Excluding Goodwill": "총 비정상 항목(영업권 제외)",
    "Net Income From Continuing Operation Net Minority Interest": "계속영업 순이익(비지배지분 제외)",
    "Reconciled Depreciation": "조정 감가상각비",
    "Reconciled Cost Of Revenue": "조정 매출원가",
    "EBITDA": "EBITDA",
    "EBIT": "영업이익(EBIT)",
    "Net Interest Income": "순이자수익",
    "Interest Expense": "이자비용",
    "Interest Income": "이자수익",
    "Normalized Income": "정상화 순이익",
    "Net Income From Continuing And Discontinued Operation": "계속·중단영업 순이익",
    "Total Expenses": "총비용",
    "Total Operating Income As Reported": "보고된 총영업이익",
    "Diluted Average Shares": "희석평균주식수",
    "Basic Average Shares": "기본평균주식수",
    "Diluted EPS": "희석주당순이익",
    "Basic EPS": "기본주당순이익",
    "Diluted NI Availto Com Stockholders": "희석 순이익(보통주주 귀속)",
    "Net Income Common Stockholders": "보통주주 순이익",
    "Net Income": "순이익",
    "Net Income Including Noncontrolling Interests": "순이익(비지배지분 포함)",
    "Net Income Continuous Operations": "계속영업 순이익",
    "Tax Provision": "법인세 비용",
    "Pretax Income": "세전이익",
    "Other Income Expense": "기타수익/비용",
    "Other Non Operating Income Expenses": "영업외 기타수익/비용",
    "Special Income Charges": "특별손익",
    "Restructuring And Mergern Acquisition": "구조조정 및 M&A 비용",
    "Net Non Operating Interest Income Expense": "영업외 순이자수익/비용",
    "Interest Expense Non Operating": "영업외 이자비용",
    "Interest Income Non Operating": "영업외 이자수익",
    "Operating Income": "영업이익",
    "Operating Expense": "영업비용",
    "Research And Development": "연구개발비",
    "Selling General And Administration": "판매관리비",
    "Gross Profit": "매출총이익",
    "Cost Of Revenue": "매출원가",
    "Total Revenue": "총매출",
    "Operating Revenue": "영업수익"
}

### 1. US데이터 수집

In [3]:
# 중간 저장용 함수
def save_chunk(data, chunk_idx, where):
    with open(f'pkl_{where}/{where}_chunk_{chunk_idx}.pkl', 'wb') as f:
        pickle.dump(data, f)


def get_data_from_yf(list_tickers, where):
    chunk_size = 100
    for chunk_idx in range(0, len(list_tickers), chunk_size):
        chunk = list_nasdaq_stocks[chunk_idx:chunk_idx+chunk_size]
        chunk_data = {}

        for ticker in tqdm(chunk, desc=f'Processing chunk {chunk_idx//chunk_size + 1}'):
            retries = 2
            while retries > 0:
                try:
                    tkr = yf.Ticker(ticker)

                    # 기본 재무제표, 배당 등
                    balance_sheet = tkr.quarterly_balance_sheet.T
                    financials = tkr.quarterly_financials.T
                    dividends = tkr.dividends
                    info = tkr.info

                    chunk_data[ticker] = {
                        'balance_sheet': balance_sheet,
                        'financials': financials,
                        'dividends': dividends,
                        'info': info
                    }

                    break
                except Exception as e:
                    print(f"{ticker} failed with error: {e}, retries left: {retries-1}")
                    retries -= 1
                    time.sleep(1)
            time.sleep(0.1)

        # chunk 저장
        save_chunk(chunk_data, chunk_idx//chunk_size + 1, where)
        #print(f"{where} download done!!")

In [4]:
nasdaq_stocks = fdr.StockListing('NASDAQ')
nyse_stocks = fdr.StockListing('NYSE')
all_stocks = pd.concat([nasdaq_stocks, nyse_stocks])

100%|██████████| 2745/2745 [00:03<00:00, 854.67it/s]


In [5]:
# list_nasdaq_stocks = nasdaq_stocks['Symbol'].tolist()
# get_data_from_yf(list_nasdaq_stocks, 'nasdaq')

In [6]:
# list_nyse_stocks = nyse_stocks['Symbol'].tolist()
# get_data_from_yf(list_nyse_stocks, 'nyse')

In [7]:
all_data = {}
folder_nasdaq_path = "./pkl_nasdaq"
folder_nyse_path = "./pkl_nyse"
files_nasdaq = [f for f in os.listdir(folder_nasdaq_path) if os.path.isfile(os.path.join(folder_nasdaq_path, f))]
files_nyse = [f for f in os.listdir(folder_nyse_path) if os.path.isfile(os.path.join(folder_nyse_path, f))]

for file_nm in tqdm(files_nasdaq, desc=f'Processing {len(folder_nasdaq_path)} files'):
    dict_file_nasdaq = pd.read_pickle(os.path.join(folder_nasdaq_path, file_nm))
    all_data.update(dict_file_nasdaq)

for file_nm in tqdm(files_nyse, desc=f'Processing {len(folder_nyse_path)} files'):
    dict_file_nyse = pd.read_pickle(os.path.join(folder_nyse_path, file_nm))
    all_data.update(dict_file_nyse)

with open(f'yf_chunk_all.pkl', 'wb') as f:
    pickle.dump(all_data, f)


Processing 10 files: 100%|██████████| 28/28 [00:00<00:00, 33.35it/s]


### 2. 티커 분류

In [8]:
dict_us_stocks = pd.read_pickle('yf_chunk_all.pkl')

list_bs_fin_tickers, list_bs_no_fin_tickers = [], []
list_no_bs_fin_tickers, list_no_bs_ca_fin_tickers = [], []
list_no_bs_no_fin_tickers = []

for ticker in dict_us_stocks.keys():
    bs = dict_us_stocks[ticker]['balance_sheet']
    financials = dict_us_stocks[ticker]['financials']

    if bs.empty or len(bs) == 0: continue
    if financials.empty or len(financials) == 0: continue

    set_core_bs_cols = set(["Total Assets", "Total Non Current Assets", "Current Assets", "Inventory"])
    set_core_fin_cols = set(["Total Revenue", "Cost Of Revenue", "Gross Profit", "Operating Income", "Net Income"])
    set_mtch_bs_cols = set([ col for col in bs.columns.tolist() for bs_col in set_core_bs_cols if col == bs_col ])
    set_mtch_fin_cols = set([ col for col in financials.columns.tolist() for fin_col in set_core_fin_cols if col == fin_col ])

    if len(set_core_bs_cols & set_mtch_bs_cols) == len(set_core_bs_cols):
        if len(set_core_fin_cols & set_mtch_fin_cols) == len(set_core_fin_cols):
            list_bs_fin_tickers.append(ticker) ## 제약, 의료장비
        else:
            list_bs_no_fin_tickers.append(ticker) ## 방송, 연구, cost_of_revenue 없음

    else: # 유동자산이 있는냐? 없는냐? 다들 재고자산은 없음
        if 'Current Assets' in set_mtch_bs_cols:
            if len(set_core_fin_cols & set_mtch_fin_cols) == len(set_core_fin_cols):
                list_no_bs_fin_tickers.append(ticker) ## 소프트웨어
            else:
                list_no_bs_ca_fin_tickers.append(ticker) ## 매출원가, 총매출이익이 없음 (생명공학, 투자지주)
        else:
            list_no_bs_no_fin_tickers.append(ticker) ##매출원가, 총매출이익이 없음 (은행)


print(  len(dict_us_stocks.keys()), len(list_bs_fin_tickers), len(list_bs_no_fin_tickers),
        len(list_no_bs_fin_tickers), len(list_no_bs_ca_fin_tickers), len(list_no_bs_no_fin_tickers))

3773 1432 16 661 729 387


### 3. 가치 기반으로 후출보 추출

- 제무재표와 손익계산서에 모두 컬럼들이 포함

In [98]:
dict_bs_fin_tickers_report = {
    'Symbol':[], 'eps':[], 'pbs':[], 'per':[], 'pbr':[], 'peg':[], 'price':[],
    'count_debt_over_200':[], 'count_current_over_100':[], 'count_quick_over_60':[],
    'avg_total_revenue':[], 'is_inc_total_revenue':[], 'is_inc_net_income':[],
    'avg_total_revenue_growth':[], 'avg_gross_profit_ratio':[], 'avg_op_income_ratio':[], 'avg_net_income_ratio':[],
    'avg_research':[], 'is_delete_share_issued':[], 'is_inc_retained_earnings':[], 'is_inc_dividends':[]
}

for ticker in list_bs_fin_tickers:
    bs = dict_us_stocks[ticker]['balance_sheet']
    financials = dict_us_stocks[ticker]['financials']
    dividends = dict_us_stocks[ticker]['dividends']
    info = dict_us_stocks[ticker]['info']

    if bs.empty or len(bs) == 0: continue
    if financials.empty or len(financials) == 0: continue
    if dividends.empty: continue

    eps = info.get('trailingEps')
    pbs = info.get('bookValue')
    per = info.get("trailingPE")
    pbr = info.get("priceToBook")
    currentPrice = info.get('currentPrice')

    if pd.isna(per) | (per == 'Infinity'): continue

    df_base = bs.join(financials)
    df_base.dropna(inplace=True, how='all')
    df_base = df_base.iloc[:5]

    df_base.loc[:, 'Total Liabilities'] = df_base['Total Non Current Liabilities Net Minority Interest'] + df_base['Current Liabilities']
    df_base.loc[:, 'Debt_Ratio'] = np.round(df_base['Total Liabilities'] / df_base["Stockholders Equity"] * 100, 2)
    df_base.loc[:, 'Current_Ratio'] = np.round(df_base["Current Liabilities"] / df_base["Current Assets"] * 100, 2)
    df_base.loc[:, 'Quick_Ratio'] = np.round(df_base["Current Liabilities"] / (df_base["Current Assets"] - df_base["Inventory"])  * 100, 2)
    count_Debt_over_200 = (df_base['Debt_Ratio'] > 200).sum()
    count_Current_over_100 = (df_base['Current_Ratio'] > 100).sum()
    count_Quick_over_60 = (df_base['Quick_Ratio'] > 60).sum()
    is_delete_share_issued = (df_base['Share Issued'] > df_base['Share Issued'].shift(1))[1:].all() if 'Share Issued' in df_base.columns else 0
    is_inc_retained_earnings = (df_base['Retained Earnings'] < df_base['Retained Earnings'].shift(1))[1:].all() if 'Retained Earnings' in df_base.columns else 0
    is_existed_research = 1 if 'Research And Development' in df_base.columns else 0

    # 총매출, 매출총이익, 영업이익, 순이익
    df_base['Total Revenue_next'] = df_base['Total Revenue'].shift(1)
    df_base.loc[:, 'Tot_Rev_Growth'] = np.round((df_base['Total Revenue_next'] - df_base['Total Revenue']) / df_base['Total Revenue'] * 100, 2)
    df_base.loc[:, 'Gross_Prf_Ratio'] = np.round(df_base['Gross Profit'] / df_base['Total Revenue'] * 100, 2)
    df_base.loc[:, 'Op_Icm_Ratio'] = np.round(df_base['Operating Income'] / df_base['Total Revenue'] * 100, 2)
    df_base.loc[:, 'Net_Icm_Ratio'] = np.round(df_base['Net Income'] / df_base['Total Revenue'] * 100, 2)

    #PEG
    peg = 0
    if 'Diluted EPS' in df_base.columns:
        diluted_eps = [v for v in financials['Diluted EPS'].values if not pd.isna(v) ]
        eps_arr = np.array(diluted_eps, dtype=float)  # 확실히 float
        if np.any(eps_arr <= 0) or np.isnan(eps_arr).any():
            eps_growth = np.nan  # 계산 불가
        else:
            eps_growth = (np.power(eps_arr[0] / eps_arr[-1], 1/(len(eps_arr)-1)) - 1) * 100
            peg = per / eps_growth if eps_growth > 0 else 0

    avg_total_revenue = df_base['Total Revenue'].mean()
    avg_total_revenue_growth = df_base['Tot_Rev_Growth'].mean()
    avg_gross_profit_ratio = df_base['Gross_Prf_Ratio'].mean()
    avg_op_income_ratio = df_base['Op_Icm_Ratio'].mean()
    avg_net_income_ratio = df_base['Net_Icm_Ratio'].mean()
    is_inc_total_revenue = (df_base['Total Revenue'] < df_base['Total Revenue'].shift(1))[1:].all()
    is_inc_net_income = (df_base['Net Income'] < df_base['Net Income'].shift(1))[1:].all()

    # 연구개발비
    avg_research = 0
    if is_existed_research:
        avg_research = np.mean(df_base['Research And Development'])

    # 배당금
    df_dividend = dividends.to_frame(name='dividend').reset_index()
    is_dividend_inc = (df_dividend['dividend'] >= df_dividend['dividend'].shift(1))[1:].all()

    list_result = [
        ticker, eps, pbs, per, pbr, peg, currentPrice, count_Debt_over_200, count_Current_over_100, count_Quick_over_60,
        avg_total_revenue, is_inc_total_revenue, is_inc_net_income,
        avg_total_revenue_growth, avg_gross_profit_ratio, avg_op_income_ratio, avg_net_income_ratio,
        avg_research, is_delete_share_issued, is_inc_retained_earnings, is_dividend_inc
    ]

    for k, v in zip(list(dict_bs_fin_tickers_report.keys()), list_result):
        dict_bs_fin_tickers_report[k].append(v)



In [99]:
df_bs_fin_report = pd.DataFrame(dict_bs_fin_tickers_report)
df_bs_fin_report[(df_bs_fin_report['is_inc_total_revenue'] == True) & (df_bs_fin_report['is_inc_dividends'] == True) &
                 (df_bs_fin_report['count_debt_over_200'] == 0) & (df_bs_fin_report['count_current_over_100'] == 0)]

,Symbol,eps,pbs,per,pbr,peg,price,count_debt_over_200,count_current_over_100,count_quick_over_60,avg_total_revenue,is_inc_total_revenue,is_inc_net_income,avg_total_revenue_growth,avg_gross_profit_ratio,avg_op_income_ratio,avg_net_income_ratio,avg_research,is_delete_share_issued,is_inc_retained_earnings,is_inc_dividends
101,AGYS,0.8,10.6,147.9,11.8,2.1,125.7,0,0,5,73617200.0,True,True,3.8,62.1,9.7,6.8,16594000.0,False,True,True
133,CAMT,1.0,12.6,103.2,8.3,0.0,104.3,0,0,0,119516200.0,True,False,2.9,50.2,26.4,14.3,11086000.0,False,False,True
141,NVDA,4.0,4.9,44.6,36.8,3.3,180.3,0,0,0,44444800000.0,True,False,13.0,70.8,59.3,53.3,4017800000.0,True,True,True
153,LRCX,4.5,8.1,34.2,19.2,3.6,155.1,0,0,3,4751952800.0,True,False,6.3,49.0,32.4,29.0,534566600.0,False,True,True
248,BELFB,5.0,33.8,30.8,4.5,0.0,152.7,0,0,1,154602800.0,True,False,9.9,38.1,13.4,9.1,7050400.0,False,False,True


- 유동자산은 있지만 재고자산과 매출총이익 없는 회사

In [101]:
dict_no_bs_ca_fin_tickers_report = {
    'Symbol':[], 'eps':[], 'pbs':[], 'per':[], 'pbr':[], 'peg':[], 'price':[],
    'count_debt_over_200':[], 'count_current_over_100':[], #'count_quick_over_60':[],
    'avg_total_revenue':[], 'is_inc_total_revenue':[], 'is_inc_net_income':[],
    'avg_total_revenue_growth':[], #'avg_gross_profit_ratio':[], 'avg_op_income_ratio':[],
    'avg_net_income_ratio':[],
    'avg_research':[], 'is_delete_share_issued':[], 'is_inc_retained_earnings':[], 'is_inc_dividends':[]
}

for ticker in list_no_bs_ca_fin_tickers:
    bs = dict_us_stocks[ticker]['balance_sheet']
    financials = dict_us_stocks[ticker]['financials']
    dividends = dict_us_stocks[ticker]['dividends']
    info = dict_us_stocks[ticker]['info']

    if bs.empty or len(bs) == 0: continue
    if financials.empty or len(financials) == 0: continue
    if dividends.empty: continue

    eps = info.get('trailingEps')
    pbs = info.get('bookValue')
    per = info.get("trailingPE")
    pbr = info.get("priceToBook")
    currentPrice = info.get('currentPrice')

    if pd.isna(per) | (per == 'Infinity'): continue

    df_base = bs.join(financials)
    df_base.dropna(inplace=True, how='all')
    df_base = df_base.iloc[:5]

    df_base.loc[:, 'Total Liabilities'] = df_base['Total Non Current Liabilities Net Minority Interest'] + df_base['Current Liabilities']
    df_base.loc[:, 'Debt_Ratio'] = np.round(df_base['Total Liabilities'] / df_base["Stockholders Equity"] * 100, 2)
    df_base.loc[:, 'Current_Ratio'] = np.round(df_base["Current Liabilities"] / df_base["Current Assets"] * 100, 2)
  #  df_base.loc[:, 'Quick_Ratio'] = np.round(df_base["Current Liabilities"] / (df_base["Current Assets"] - df_base["Inventory"])  * 100, 2)
    count_Debt_over_200 = (df_base['Debt_Ratio'] > 200).sum()
    count_Current_over_100 = (df_base['Current_Ratio'] > 100).sum()
  #  count_Quick_over_60 = (df_base['Quick_Ratio'] > 60).sum()
    is_delete_share_issued = (df_base['Share Issued'] > df_base['Share Issued'].shift(1))[1:].all() if 'Share Issued' in df_base.columns else 0
    is_inc_retained_earnings = (df_base['Retained Earnings'] < df_base['Retained Earnings'].shift(1))[1:].all() if 'Retained Earnings' in df_base.columns else 0
    is_existed_research = 1 if 'Research And Development' in df_base.columns else 0

    # 총매출, 매출총이익, 영업이익, 순이익
    df_base['Total Revenue_next'] = df_base['Total Revenue'].shift(1)
    df_base.loc[:, 'Tot_Rev_Growth'] = np.round((df_base['Total Revenue_next'] - df_base['Total Revenue']) / df_base['Total Revenue'] * 100, 2)
    #df_base.loc[:, 'Gross_Prf_Ratio'] = np.round(df_base['Gross Profit'] / df_base['Total Revenue'] * 100, 2)
    #df_base.loc[:, 'Op_Icm_Ratio'] = np.round(df_base['Operating Income'] / df_base['Total Revenue'] * 100, 2)
    df_base.loc[:, 'Net_Icm_Ratio'] = np.round(df_base['Net Income'] / df_base['Total Revenue'] * 100, 2)

    #PEG
    peg = 0
    if 'Diluted EPS' in df_base.columns:
        diluted_eps = [v for v in financials['Diluted EPS'].values if not pd.isna(v) ]
        eps_arr = np.array(diluted_eps, dtype=float)  # 확실히 float
        if np.any(eps_arr <= 0) or np.isnan(eps_arr).any():
            eps_growth = np.nan  # 계산 불가
        else:
            eps_growth = (np.power(eps_arr[0] / eps_arr[-1], 1/(len(eps_arr)-1)) - 1) * 100
            peg = per / eps_growth

    avg_total_revenue = df_base['Total Revenue'].mean()
    avg_total_revenue_growth = df_base['Tot_Rev_Growth'].mean()
    #avg_gross_profit_ratio = df_base['Gross_Prf_Ratio'].mean()
    #avg_op_income_ratio = df_base['Op_Icm_Ratio'].mean()
    avg_net_income_ratio = df_base['Net_Icm_Ratio'].mean()
    is_inc_total_revenue = (df_base['Total Revenue'] < df_base['Total Revenue'].shift(1))[1:].all()
    is_inc_net_income = (df_base['Net Income'] < df_base['Net Income'].shift(1))[1:].all()

    # 연구개발비
    avg_research = 0
    if is_existed_research:
        avg_research = np.mean(df_base['Research And Development'])

    # 배당금
    df_dividend = dividends.to_frame(name='dividend').reset_index()
    is_dividend_inc = (df_dividend['dividend'] >= df_dividend['dividend'].shift(1))[1:].all()

    list_result = [
        ticker, eps, pbs, per, pbr, peg, currentPrice, count_Debt_over_200, count_Current_over_100, #count_Quick_over_60,
        avg_total_revenue, is_inc_total_revenue, is_inc_net_income,
        avg_total_revenue_growth, #avg_gross_profit_ratio, avg_op_income_ratio,
        avg_net_income_ratio,
        avg_research, is_delete_share_issued, is_inc_retained_earnings, is_dividend_inc
    ]

    for k, v in zip(list(dict_no_bs_ca_fin_tickers_report.keys()), list_result):
        dict_no_bs_ca_fin_tickers_report[k].append(v)



In [104]:
df_no_bs_ca_fin_report = pd.DataFrame(dict_no_bs_ca_fin_tickers_report)
df_no_bs_ca_fin_report[(df_no_bs_ca_fin_report['is_inc_net_income'] == True) & (df_no_bs_ca_fin_report['is_inc_dividends'] == True) ]

,Symbol,eps,pbs,per,pbr,peg,price,count_debt_over_200,count_current_over_100,avg_total_revenue,is_inc_total_revenue,is_inc_net_income,avg_total_revenue_growth,avg_net_income_ratio,avg_research,is_delete_share_issued,is_inc_retained_earnings,is_inc_dividends


- 유동자산, 재고자산 없고 매출총이익, 영업이익 없는 회사

In [106]:
dict_no_bs_no_fin_tickers_report = {
    'Symbol':[], 'eps':[], 'pbs':[], 'per':[], 'pbr':[], 'peg':[], 'price':[],
    #'count_debt_over_200':[], 'count_current_over_100':[], 'count_quick_over_60':[],
    'avg_total_revenue':[], 'is_inc_total_revenue':[], 'is_inc_net_income':[],
    'avg_total_revenue_growth':[], #'avg_gross_profit_ratio':[], 'avg_op_income_ratio':[],
    'avg_net_income_ratio':[],
    'avg_research':[], 'is_delete_share_issued':[], 'is_inc_retained_earnings':[], 'is_inc_dividends':[]
}

for ticker in list_no_bs_no_fin_tickers:
    bs = dict_us_stocks[ticker]['balance_sheet']
    financials = dict_us_stocks[ticker]['financials']
    dividends = dict_us_stocks[ticker]['dividends']
    info = dict_us_stocks[ticker]['info']

    if bs.empty or len(bs) == 0: continue
    if financials.empty or len(financials) == 0: continue
    if dividends.empty: continue

    eps = info.get('trailingEps')
    pbs = info.get('bookValue')
    per = info.get("trailingPE")
    pbr = info.get("priceToBook")
    currentPrice = info.get('currentPrice')

    if pd.isna(per) | (per == 'Infinity'): continue

    df_base = bs.join(financials)
    df_base.dropna(inplace=True, how='all')
    df_base = df_base.iloc[:5]

  #  df_base.loc[:, 'Total Liabilities'] = df_base['Total Non Current Liabilities Net Minority Interest'] + df_base['Current Liabilities']
  #  df_base.loc[:, 'Debt_Ratio'] = np.round(df_base['Total Liabilities'] / df_base["Stockholders Equity"] * 100, 2)
  #  df_base.loc[:, 'Current_Ratio'] = np.round(df_base["Current Liabilities"] / df_base["Current Assets"] * 100, 2)
  #  df_base.loc[:, 'Quick_Ratio'] = np.round(df_base["Current Liabilities"] / (df_base["Current Assets"] - df_base["Inventory"])  * 100, 2)
  #  count_Debt_over_200 = (df_base['Debt_Ratio'] > 200).sum()
  #  count_Current_over_100 = (df_base['Current_Ratio'] > 100).sum()
  #  count_Quick_over_60 = (df_base['Quick_Ratio'] > 60).sum()
    is_delete_share_issued = (df_base['Share Issued'] > df_base['Share Issued'].shift(1))[1:].all() if 'Share Issued' in df_base.columns else 0
    is_inc_retained_earnings = (df_base['Retained Earnings'] < df_base['Retained Earnings'].shift(1))[1:].all() if 'Retained Earnings' in df_base.columns else 0
    is_existed_research = 1 if 'Research And Development' in df_base.columns else 0

    # 총매출, 매출총이익, 영업이익, 순이익
    df_base['Total Revenue_next'] = df_base['Total Revenue'].shift(1)
    df_base.loc[:, 'Tot_Rev_Growth'] = np.round((df_base['Total Revenue_next'] - df_base['Total Revenue']) / df_base['Total Revenue'] * 100, 2)
    #df_base.loc[:, 'Gross_Prf_Ratio'] = np.round(df_base['Gross Profit'] / df_base['Total Revenue'] * 100, 2)
    #df_base.loc[:, 'Op_Icm_Ratio'] = np.round(df_base['Operating Income'] / df_base['Total Revenue'] * 100, 2)
    df_base.loc[:, 'Net_Icm_Ratio'] = np.round(df_base['Net Income'] / df_base['Total Revenue'] * 100, 2)

    #PEG
    peg = 0
    if 'Diluted EPS' in df_base.columns:
        diluted_eps = [v for v in financials['Diluted EPS'].values if not pd.isna(v) ]
        eps_arr = np.array(diluted_eps, dtype=float)  # 확실히 float
        if np.any(eps_arr <= 0) or np.isnan(eps_arr).any():
            eps_growth = np.nan  # 계산 불가
        else:
            eps_growth = (np.power(eps_arr[0] / eps_arr[-1], 1/(len(eps_arr)-1)) - 1) * 100
            peg = per / eps_growth if eps_growth > 0 else 0

    avg_total_revenue = df_base['Total Revenue'].mean()
    avg_total_revenue_growth = df_base['Tot_Rev_Growth'].mean()
    #avg_gross_profit_ratio = df_base['Gross_Prf_Ratio'].mean()
    #avg_op_income_ratio = df_base['Op_Icm_Ratio'].mean()
    avg_net_income_ratio = df_base['Net_Icm_Ratio'].mean()
    is_inc_total_revenue = (df_base['Total Revenue'] < df_base['Total Revenue'].shift(1))[1:].all()
    is_inc_net_income = (df_base['Net Income'] < df_base['Net Income'].shift(1))[1:].all()

    # 연구개발비
    avg_research = 0
    if is_existed_research:
        avg_research = np.mean(df_base['Research And Development'])

    # 배당금
    df_dividend = dividends.to_frame(name='dividend').reset_index()
    is_dividend_inc = (df_dividend['dividend'] >= df_dividend['dividend'].shift(1))[1:].all()

    list_result = [
        ticker, eps, pbs, per, pbr, peg, currentPrice, #count_Debt_over_200, count_Current_over_100, count_Quick_over_60,
        avg_total_revenue, is_inc_total_revenue, is_inc_net_income,
        avg_total_revenue_growth, #avg_gross_profit_ratio, avg_op_income_ratio,
        avg_net_income_ratio,
        avg_research, is_delete_share_issued, is_inc_retained_earnings, is_dividend_inc
    ]

    for k, v in zip(list(dict_no_bs_no_fin_tickers_report.keys()), list_result):
        dict_no_bs_no_fin_tickers_report[k].append(v)



In [107]:
df_no_bs_no_fin_report = pd.DataFrame(dict_no_bs_no_fin_tickers_report)
df_no_bs_no_fin_report[(df_no_bs_no_fin_report['is_inc_net_income'] == True) & (df_no_bs_no_fin_report['is_inc_dividends'] == True) ]
                      # & (df_no_bs_no_fin_report['per'] <= 10) & (df_no_bs_no_fin_report['pbr'] <= 1) ]

,Symbol,eps,pbs,per,pbr,peg,price,avg_total_revenue,is_inc_total_revenue,is_inc_net_income,avg_total_revenue_growth,avg_net_income_ratio,avg_research,is_delete_share_issued,is_inc_retained_earnings,is_inc_dividends
58,TCBS,0.9,19.5,18.5,0.8,1.0,15.9,3924200.0,False,True,4.4,15.5,0,False,True,True
63,HBCP,5.6,54.0,9.9,1.0,1.3,55.8,35991600.0,True,True,2.7,29.8,0,False,True,True
100,BWFG,3.7,38.1,12.6,1.2,0.2,46.0,24207400.0,False,True,7.0,24.5,0,False,True,True
128,INTR,0.5,22.0,17.4,0.4,1.4,8.9,1904735600.0,False,True,6.7,15.3,0,False,0,True
223,JMSB,1.4,18.3,14.4,1.1,2.4,20.0,14881800.0,True,True,4.2,32.7,0,False,True,True
288,FULTP,1.6,17.8,12.3,1.1,0.0,19.3,325123200.0,False,True,0.7,26.1,0,False,True,True


- 유동,재고자산 없으면서 손익계산서 있는 회사

In [108]:
dict_no_bs_fin_tickers_report = {
    'Symbol':[], 'eps':[], 'pbs':[], 'per':[], 'pbr':[], 'peg':[], 'price':[],
    'count_debt_over_200':[],# 'count_current_over_100':[], 'count_quick_over_60':[],
    'avg_total_revenue':[], 'is_inc_total_revenue':[], 'is_inc_net_income':[],
    'avg_total_revenue_growth':[], 'avg_gross_profit_ratio':[], 'avg_op_income_ratio':[], 'avg_net_income_ratio':[],
    'avg_research':[], 'is_delete_share_issued':[], 'is_inc_retained_earnings':[], 'is_inc_dividends':[]
}

for ticker in list_no_bs_fin_tickers:
    bs = dict_us_stocks[ticker]['balance_sheet']
    financials = dict_us_stocks[ticker]['financials']
    dividends = dict_us_stocks[ticker]['dividends']
    info = dict_us_stocks[ticker]['info']

    if bs.empty or len(bs) == 0: continue
    if financials.empty or len(financials) == 0: continue
    if dividends.empty: continue

    eps = info.get('trailingEps')
    pbs = info.get('bookValue')
    per = info.get("trailingPE")
    pbr = info.get("priceToBook")
    currentPrice = info.get('currentPrice')

    if pd.isna(per) | (per == 'Infinity'): continue

    df_base = bs.join(financials)
    df_base.dropna(inplace=True, how='all')
    df_base = df_base.iloc[:5]

    df_base.loc[:, 'Total Liabilities'] = df_base['Total Non Current Liabilities Net Minority Interest'] + df_base['Current Liabilities']
    df_base.loc[:, 'Debt_Ratio'] = np.round(df_base['Total Liabilities'] / df_base["Stockholders Equity"] * 100, 2)
  #  df_base.loc[:, 'Current_Ratio'] = np.round(df_base["Current Liabilities"] / df_base["Current Assets"] * 100, 2)
  #  df_base.loc[:, 'Quick_Ratio'] = np.round(df_base["Current Liabilities"] / (df_base["Current Assets"] - df_base["Inventory"])  * 100, 2)
    count_Debt_over_200 = (df_base['Debt_Ratio'] > 200).sum()
  #  count_Current_over_100 = (df_base['Current_Ratio'] > 100).sum()
  #  count_Quick_over_60 = (df_base['Quick_Ratio'] > 60).sum()
    is_delete_share_issued = (df_base['Share Issued'] > df_base['Share Issued'].shift(1))[1:].all() if 'Share Issued' in df_base.columns else 0
    is_inc_retained_earnings = (df_base['Retained Earnings'] < df_base['Retained Earnings'].shift(1))[1:].all() if 'Retained Earnings' in df_base.columns else 0
    is_existed_research = 1 if 'Research And Development' in df_base.columns else 0

    # 총매출, 매출총이익, 영업이익, 순이익
    df_base['Total Revenue_next'] = df_base['Total Revenue'].shift(1)
    df_base.loc[:, 'Tot_Rev_Growth'] = np.round((df_base['Total Revenue_next'] - df_base['Total Revenue']) / df_base['Total Revenue'] * 100, 2)
    df_base.loc[:, 'Gross_Prf_Ratio'] = np.round(df_base['Gross Profit'] / df_base['Total Revenue'] * 100, 2)
    df_base.loc[:, 'Op_Icm_Ratio'] = np.round(df_base['Operating Income'] / df_base['Total Revenue'] * 100, 2)
    df_base.loc[:, 'Net_Icm_Ratio'] = np.round(df_base['Net Income'] / df_base['Total Revenue'] * 100, 2)

    #PEG
    peg = 0
    if 'Diluted EPS' in df_base.columns:
        diluted_eps = [v for v in financials['Diluted EPS'].values if not pd.isna(v) ]
        eps_arr = np.array(diluted_eps, dtype=float)  # 확실히 float
        if np.any(eps_arr <= 0) or np.isnan(eps_arr).any():
            eps_growth = np.nan  # 계산 불가
        else:
            eps_growth = (np.power(eps_arr[0] / eps_arr[-1], 1/(len(eps_arr)-1)) - 1) * 100
            peg = per / eps_growth if eps_growth > 0 else 0

    avg_total_revenue = df_base['Total Revenue'].mean()
    avg_total_revenue_growth = df_base['Tot_Rev_Growth'].mean()
    avg_gross_profit_ratio = df_base['Gross_Prf_Ratio'].mean()
    avg_op_income_ratio = df_base['Op_Icm_Ratio'].mean()
    avg_net_income_ratio = df_base['Net_Icm_Ratio'].mean()
    is_inc_total_revenue = (df_base['Total Revenue'] < df_base['Total Revenue'].shift(1))[1:].all()
    is_inc_net_income = (df_base['Net Income'] < df_base['Net Income'].shift(1))[1:].all()

    # 연구개발비
    avg_research = 0
    if is_existed_research:
        avg_research = np.mean(df_base['Research And Development'])

    # 배당금
    df_dividend = dividends.to_frame(name='dividend').reset_index()
    is_dividend_inc = (df_dividend['dividend'] >= df_dividend['dividend'].shift(1))[1:].all()

    list_result = [
        ticker, eps, pbs, per, pbr, peg, currentPrice, count_Debt_over_200, #count_Current_over_100, count_Quick_over_60,
        avg_total_revenue, is_inc_total_revenue, is_inc_net_income,
        avg_total_revenue_growth, avg_gross_profit_ratio, avg_op_income_ratio, avg_net_income_ratio,
        avg_research, is_delete_share_issued, is_inc_retained_earnings, is_dividend_inc
    ]

    for k, v in zip(list(dict_no_bs_fin_tickers_report.keys()), list_result):
        dict_no_bs_fin_tickers_report[k].append(v)



In [109]:
df_no_bs_fin_report = pd.DataFrame(dict_no_bs_fin_tickers_report)
df_no_bs_fin_report[(df_no_bs_fin_report['is_inc_net_income'] == True) & (df_no_bs_fin_report['is_inc_dividends'] == True) ]
                      # & (df_no_bs_no_fin_report['per'] <= 10) & (df_no_bs_no_fin_report['pbr'] <= 1) ]

,Symbol,eps,pbs,per,pbr,peg,price,count_debt_over_200,avg_total_revenue,is_inc_total_revenue,is_inc_net_income,avg_total_revenue_growth,avg_gross_profit_ratio,avg_op_income_ratio,avg_net_income_ratio,avg_research,is_delete_share_issued,is_inc_retained_earnings,is_inc_dividends
22,GDEV,2.6,-5.8,7.2,-3.2,0.7,18.5,0,112867500.0,True,True,13.3,66.8,15.1,13.9,0.0,False,False,True
76,TW,2.9,29.4,36.8,3.7,2.9,107.9,0,488701000.0,False,True,3.3,66.3,39.6,30.3,0.0,False,True,True
82,FUTU,8.9,261.6,19.0,0.6,0.8,169.6,5,4855421400.0,True,True,17.1,84.7,56.3,45.1,437265200.0,False,False,True
